In [ ]:
import org.apache.spark.sql.functions.input_file_name
import scala.sys.process.Process
import sys.process._
import java.io.File;
import scala.Byte
import scala.util.Random
import java.nio.file.{Files, Paths}
import java.nio.ByteBuffer
import java.util.Arrays 
import java.nio.ByteOrder
import java.io.RandomAccessFile
import java.io.PrintWriter
import java.io.FileWriter
import scala.collection.mutable.ListBuffer
import java.lang.Math.toIntExact
import scala.reflect.ClassTag
import sys.process._
import geotrellis.raster._
import geotrellis.proj4.CRS
import geotrellis.raster.io.geotiff.writer.GeoTiffWriter
import geotrellis.raster.io.geotiff.{SinglebandGeoTiff, _}
import geotrellis.raster.{CellType, DoubleArrayTile}
import geotrellis.spark.io.hadoop._
import geotrellis.vector.{Extent, ProjectedExtent}
import org.apache.spark.mllib.linalg.Vector
import org.apache.spark.rdd.RDD
import org.apache.spark.{SparkConf, SparkContext}
import org.apache.spark.input.PortableDataStream
import scala.collection.immutable.Map
import org.apache.commons.io.FilenameUtils;
import org.apache.spark.sql.functions.input_file_name
import scala.sys.process.Process
import sys.process._
import scala.Byte
import java.nio.file.{Files, Paths}
import java.nio.ByteBuffer
import java.util.Arrays 
import java.nio.ByteOrder
import java.lang.Math.toIntExact
import scala.reflect.ClassTag
import org.apache.spark.broadcast.Broadcast
import org.apache.spark.HashPartitioner
import java.util.Calendar
import java.time.temporal.ChronoUnit
import org.apache.spark.Partitioner
import scala.io.Source

## Control

In [ ]:
//CONTROL PARAMETERS (NDVI and EVI)
//Tilling 
var startRow = 1 
var endRow = 3000
var starColumn = 1
var endColumn = 6500
var numberTilesRow = 32
var numberTilesColumn = 32
var number_tiles = numberTilesRow * numberTilesColumn

//Settings files  parameters
var n_year = "19"
var n_points_per_year = "36"
val settigns_dir = "/data/local/home/parrot/minio2/settigs_dir/"
var settingsNameTemplate = "template.set"
val outputSettingName = "USA_part"
val settingsList: String = "/data/local/home/parrot/minio2/listfile_cluster.txt"
val executables_dir =  "/data/local/home/parrot/minio2/executables/"
val executable_path = executables_dir + "TSF_process.x64"

var filepath = "hdfs:///user/hadoop/evi_new_template.tif"

//Change here parameters for outpt paths
var results_folder = "performance_testing/"
var results_folderSOS = "performance_testing/"

var save_path = "/data/local/home/parrot/minio2/" + results_folder
var save_path_sos = "/data/local/home/parrot/minio2/" + results_folderSOS
var save_path_hdfs = "hdfs:///user/spark/" + results_folder



## Create Settigns Files

In [ ]:
def CreateSettigsFilesForEachNode() : Unit = {
    
    val rowRange = endRow - startRow
    var rowCounter = startRow
    
    var columnRange = endColumn - starColumn
    var columnCounter = starColumn

    var settingFile = new File("/data/local/home/parrot/minio2/" + settingsNameTemplate) 
//     val settingFile = new File("/data/local/home/parrot/minio2/" + settingsNameTemplate) 
    val fw_list = new FileWriter(settingsList)

    val lines = scala.io.Source.fromFile(settingFile).mkString.split("\\r?\\n")
    var endRowInc =startRow
    var endColumnInc =starColumn
    var tile_number = 1
    
    
    for( rowTile <- 1 to numberTilesRow){
      if( rowTile != numberTilesRow) {
        endRowInc = rowCounter + rowRange / numberTilesRow
      }
      else{
        // last task, compensate for the pixel increment and non-whole division
        endRowInc = (rowCounter + rowRange  / numberTilesRow) - numberTilesRow + 1 + (rowRange % numberTilesRow)
      }
        
      for( columnTile <- 1 to numberTilesColumn){
          if( columnTile != numberTilesColumn) {
            endColumnInc = columnCounter + columnRange / numberTilesColumn
          }
          else{
            // last task, compensate for the pixel increment and non-whole division
            endColumnInc = (columnCounter + columnRange  / numberTilesColumn) - numberTilesColumn + 1 + (columnRange % numberTilesColumn)
          }
        
          //change corresponding lines         
          lines(1)= "cluster" + tile_number +"      %Job_name (no blanks)" //change job name
          lines(10) = rowCounter.toString + " "+ endRowInc.toString + " "+ columnCounter.toString+" "+ endColumnInc.toString +"      %Processing window (start row end row start col end col)"
          lines(11) = n_year + " " + n_points_per_year  + "           %No. years and no. points per year"
          columnCounter +=  columnRange /numberTilesColumn +1

          println( "Settings file: " + tile_number )
          println(lines(10))

          val settingFileOut = new File(settigns_dir + outputSettingName+"_cluster" + ".set"+ tile_number) // Temporary File
          new PrintWriter(settingFileOut){write(lines.mkString("\n"))}.close() //close file
          fw_list.write(settigns_dir + outputSettingName+"_cluster" + ".set"+ tile_number + "\t" +"cluster" + tile_number +"_TS.tpa" + "\t" +"cluster" + tile_number +"_TS.ndx"  + "\n")
          tile_number = tile_number + 1
      }
        rowCounter +=  rowRange /numberTilesRow +1
        columnCounter = starColumn
        endColumnInc =starColumn
    }
     fw_list.close() 
}
    
CreateSettigsFilesForEachNode()

## Prepare Jobs in RDD

In [ ]:
class ExactPartitioner(override val numPartitions: Int) extends Partitioner {

  def getPartition(key: Any): Int = key match {
    // `k` is assumed to go continuously from 0 to elements-1.
    case _ =>      key.asInstanceOf[Int] 
  }
}


In [ ]:

//Argumenets: settings path, output .tpa file
var settings_rdd : RDD[(String, String,String)] = sc.textFile("file://" + settingsList)
.map(str => str.split("\t"))
.map(arr => (arr(0), arr(1), arr(2)))

// Repartition on each line 
//default partitioner can assign several rdd records (block_ids) to one partitioner, few empty
var settings_rdd_part = settings_rdd.repartition(number_tiles).zipWithIndex.map{case (k,v) => (v.toInt,k)} 

var one_task_per_record : RDD[(String,String,String)]= settings_rdd_part.partitionBy(new ExactPartitioner(number_tiles)).values


one_task_per_record.mapPartitionsWithIndex((idx, it) => Iterator((idx, it.toList))).toDF("partition_id", "block_ids").show(2024,false)



## Execute !

In [ ]:
val res = one_task_per_record.map( setting_tile => { Seq(executable_path, setting_tile._1,"1" ).!
                               Seq("mv",setting_tile._2,save_path).!
                               Seq("mv",setting_tile._3,save_path).!})
res.collect()


## Only Run if Execution Problems, Will stop the TimeSat process from within the machines

In [ ]:
var kill = one_task_per_record.map( setting_tile => { Seq("sudo","pkill","TSF_process.x64").!})
kill.collect()

In [ ]:

//Execute to set the permissions: HADOOP_USER_NAME=root hadoop fs -chmod -R 777 /user/spark
val cmd = "hadoop fs -copyFromLocal " + save_path + " " + "/user/spark"
Process(cmd)!

In [ ]:
var data = sc.textFile("file:///data/local/home/parrot/minio2/listfileNdviNew.txt")
var count  = names.first()
var filtered_dates = data.filter(row => row != count)
var new2 = filtered_dates.map( data => data.split("\\.")).map(path => path(0).split("/")).map(path => path(path.size -1))

var new3 =new2.zipWithIndex().map { case (v, i) => (i+1, v) } //cache
var index_date  = new3.collectAsMap().map(kv => (kv._1,kv._2)).toMap

var index_dateBC = sc.broadcast(index_date)

index_date.get(28)


## Read Binary Input into rdd

In [ ]:

var rdd_tpa = sc.binaryFiles(save_path_hdfs + "/*.tpa").map(m => (FilenameUtils.getBaseName(m._1).split("_")(0), m._2))
var rdd_tpx = sc.binaryFiles(save_path_hdfs+ "/*.ndx").map(m => (FilenameUtils.getBaseName(m._1).split("_")(0), m._2))

var all :RDD[(String,(PortableDataStream,PortableDataStream))]  = rdd_tpa.join(rdd_tpx)



## Extract Seasonality Information per pixel

In [ ]:

def getSeasons(filename: String, indexStream: PortableDataStream, seasonalityStream: PortableDataStream  ): Array[Float] = {
    var index : Array[Byte] = indexStream.toArray()
    if(index.length != 0){
        var seasonality : Array[Byte] = seasonalityStream.toArray()
    //     var index_tuple : Array[Long] = new Array[Long]( index.length/8)
        var nseasons_int_def = toIntExact(ByteBuffer.wrap(index.slice(16,24)).order(ByteOrder.LITTLE_ENDIAN).getLong)
        var array = new Array[Float](index.length/32 * nseasons_int_def *16)
        var arrayIndex = 0

        //loop on the pixel positions 
        for(i <- 0 until index.length by 32){ 

            var nseasons_int = toIntExact(ByteBuffer.wrap(index.slice(i+16,i+24)).order(ByteOrder.LITTLE_ENDIAN).getLong)   // number seasons
            var position_int = toIntExact(ByteBuffer.wrap(index.slice(i+24,i+32)).order(ByteOrder.LITTLE_ENDIAN).getLong) -1 //adjust for somewhat -1 offset in the file 

            //loop on the available seasons 
            for (z <- 1 to nseasons_int){
                array(arrayIndex) = ByteBuffer.wrap(index.slice(i,i+8)).order(ByteOrder.LITTLE_ENDIAN).getLong //row
                array(arrayIndex + 1) = ByteBuffer.wrap(index.slice(i+8,i+16)).order(ByteOrder.LITTLE_ENDIAN).getLong //column
                array(arrayIndex + 2) = z //number of season we are reading
                if (z.equals(1)) position_int+= 12  //skip row,column, number seasons, we have them from index file
                //loap the  seasonality parametars
                for (m <- 1 to 13){ 
                    array(arrayIndex + 2 + m) = (ByteBuffer.wrap(seasonality.slice(position_int, position_int+4)).order(ByteOrder.LITTLE_ENDIAN).getFloat)
                    position_int+= 4
                }
                arrayIndex = arrayIndex + 3 + 13
            }
        }
        return array
      }
    else{
        return new Array[Float](16).fill(-1)
    }
}


var all_seasonality_parametars  = all.flatMap(m => { getSeasons(m._1, m._2._2, m._2._1).grouped(16).toList}).filter(m => m(0) != -1)

//here we drop the othe seasonality parameteers and get SOS only 
//tuple in the form (x,y,season,SOS) --> [(1609.0,2721.0,1.0,45.88774), (1609.0,2721.0,2.0,80.46434), ...]
var sos_tuple = all_seasonality_parametars.map(m => (m(0),m(1),m(2),m(3)))



In [ ]:
var sos_tuple_s1  = sos_tuple.filter(x => x._3 == 15.toFloat).filter(x => x._1 == 1000).map{case (x,y,season,value) => {
       
        (x,y,season,value)
    }}

sos_tuple_s1.take(1000).foreach(println)

In [ ]:
//find number of seasons //think of a better way, instead of going throug all the rdd
val n_seasons_found : Int = sos_tuple.map(x => x._3).max().toInt

## Prepare Template

In [ ]:
//Single band GeoTiff
//Since it is a single GeoTiff, it will be a RDD with a tile.
var geoTiffs : RDD[(ProjectedExtent, Tile)] = sc.hadoopGeoTiffRDD(filepath)
var proExtents_RDD : RDD[ProjectedExtent] = geoTiffs.keys
var tiles_RDD : RDD[Tile]= geoTiffs.values


var grids_RDD = tiles_RDD.map(m => m.toArrayDouble())

var bands_withIndex = grids_RDD.zipWithIndex().map { case (v, i) => (i, v) }

//Array[(Long, Double)]
var grid0 = bands_withIndex.filter(m => m._1 == 0).values.flatMap( m => m).zipWithIndex.map{case (v,i) => (i,v)}    //

var extents_withIndex = proExtents_RDD.zipWithIndex().map{case (e,v) => (v,e)} //cache
var projected_extent = (extents_withIndex.filter(m => m._1 == 0).values.collect())(0) //.cache.collect())(0)


## Align Seasons

In [ ]:
var filtered = sos_tuple.filter(x => x._3 == 1.0 && x._4 > 27).map{ case (x, y,season,value) => (x, y) }

//NDVI
var filtered2 = sos_tuple.filter(x => x._3 == 18.0 && (x._4 > 639 || x._4 == 0)).map{ case (x, y,season,value) => (x, y) }
var intersection = sc.broadcast(filtered.intersection(filtered2).collect.toSet)

var aligned_seasons_1 = sos_tuple.map{ case (x, y,season,value) => {
    if (intersection.value.contains((x,y))){
        (x, y,season + 1,value)
    }else{
    (x, y,season,value)
    }
    
}} //cache


## Generate GeoTiff for each available season

In [ ]:


var num_cols :Int = 6500
var num_rows :Int = 3000
var num_colsB = sc.broadcast(num_cols)
var start_year = 1999
var start_yearB = sc.broadcast(start_year)
var season_number = 1




for( season_number <- 1 to n_seasons_found + 1){
     
    var sos_tuple_s1  = aligned_seasons_1.filter(x => x._3 == season_number.toFloat).filter(x => x._4 != 0.0).map{case (x,y,season,value) => {
        var rounded = Math.rint( (value -1)*10.139)   //round to int by using 2 digits of precision
        var date_TimeSat : Calendar = Calendar.getInstance()
        
        var leap_year_days = 0
        if(season_number >= 2 && season_number<=5){
            leap_year_days = 1
        }
        else if (season_number >= 6 && season_number<= 9){
             leap_year_days = 2
        }
        else if (season_number >= 10 && season_number<= 13){
             leap_year_days = 3
        }
        else if (season_number >= 14 && season_number<= 17){
             leap_year_days = 4
        }else if (season_number> 17){
            leap_year_days = 5
        }   
        
        date_TimeSat.set(start_yearB.value , 0, 3)
        date_TimeSat.add(Calendar.DATE, rounded.toInt + leap_year_days)
        var day_of_year = date_TimeSat.get(Calendar.DAY_OF_YEAR)
        //offster early spings
        if(day_of_year < 270){
            (x,y,season,day_of_year)}
        else{
            (x,y,season,day_of_year- 365)}
        
    
    }}
    
    print("=====Calculating Season "+ season_number)
//      sos_tuple_s2.take(1000).foreach(println)

    //The .get(col,row) methods are implemented by using the equation cols * row + col to translate from grid coordinates to array index
    var season_to_array = sos_tuple_s1.map(x => (num_colsB.value.toInt*(x._1.toInt -1) + (x._2.toInt- 1).toLong, x._4))

    var result_2: RDD[(Long, (Double, Option[Int]))] = grid0.leftOuterJoin(season_to_array)
    // result_2.take(20).foreach(println)

    //TODO: 1. If value in original .tiff  , but None in seasonality ==> set NaN for the corresponding index
    var result_3: RDD[(Long, Double)] = result_2.map { 
      case (index, (value_org, value_pheno)) => {if (value_pheno.isDefined)  (index,value_pheno.get)
                                                 else (index, Double.NaN)  }
    }
    

    var season = DoubleArrayTile(result_3.sortBy(_._1).map(_._2).collect() , num_cols, num_rows)


    var geoTif = new SinglebandGeoTiff(season, projected_extent.extent, projected_extent.crs, Tags.empty, GeoTiffOptions(compression.DeflateCompression))
    var year = start_year + season_number -1
    var output = save_path_sos + year + ".tif"
    GeoTiffWriter.write(geoTif, output)    
 }

